In [1]:
#dependecies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc
###
from sqlalchemy import Column, Float, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
import pandas as pd
import numpy as np
import os

from flask import Flask, render_template, jsonify, request
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
engine = create_engine("sqlite:///nba.sqlite")
Base.metadata.create_all(engine)

In [3]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Print all of the classes mapped to the Base
print(Base.classes.keys())

['nba_table']


In [4]:
# Save reference to the table
nba = Base.classes.nba_table
# Create our session (link) from Python to the DB
session = Session(engine)

# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [5]:
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('nba_table')
#print(columns)

In [30]:
#collecting all years and types of games played in a year
#@app.route('/yearandtype')
def yearandtype():
    year = 0
    typ = ""
    yearType = []
    #yearAndType = {}
    results = session.query(nba.Season, nba.Type).order_by(nba.Season.desc()).distinct()
    #print(len(results))
    for result in results:
        year = result[0]
        typ = result[1]
        yearType.append(str(year)+":"+typ)
    return (yearType)

yearandtype()

['2017:Regular',
 '2017:Playoff',
 '2016:Regular',
 '2016:Playoff',
 '2015:Regular',
 '2015:Playoff',
 '2014:Regular',
 '2014:Playoff',
 '2013:Regular',
 '2013:Playoff']

In [6]:
#is there a relationship between age and average salary?
#@app.route('/avgageandsalary')
def avgageandsalary():
    age = []
    avgSalary=[]
    avgAgeAndSalary = {}
    count = 0
    salary = 0
    results = session.query(nba.AGE, nba.SAL).order_by(nba.AGE.desc()).all()
    #print(len(results))
    for i in range (0, len(results)-1):   
        if(results[i][0] ==results[i+1][0]):
            count = count+1
            salary= salary+(results[i][1])
            #print(salary)
        else:
            count = count+1
            salary= salary+(results[i][1])
            avgSalary.append(round(salary/count, 2))
            age.append(results[i][0])
            count = 0
            salary = 0
    #print(avgSalary)
    count = count+1
    salary= salary+(results[i+1][1])
    avgSalary.append(round(salary/count, 2))
    age.append(results[i][0])
    avgAgeAndSalary =dict(zip(age, avgSalary))
    #print("--")
    #print(i)
    #print("--")
    #print(count)
    #print("--")
    #print(avgAgeAndSalary)
    return avgAgeAndSalary
avgageandsalary()

{19: 2870305.71,
 20: 2361965.94,
 21: 2346180.73,
 22: 1853676.96,
 23: 2139149.93,
 24: 3287110.93,
 25: 4509029.43,
 26: 5662844.85,
 27: 6457493.81,
 28: 6552864.08,
 29: 6704963.62,
 30: 7503043.25,
 31: 7956342.4,
 32: 7335882.95,
 33: 5341421.15,
 34: 5264754.84,
 35: 4841498.67,
 36: 5252520.82,
 37: 5079460.05,
 38: 4349823.65,
 39: 6283730.59,
 40: 3867407.31}

In [34]:
#age distribution of all players in a given year -- how many players are in each age year range
year = 2017
type = 'Regular'
#@app.route('/ageDistribution/<year>/<type>')
def ageDistribution(year, typ):
    age =0
    names = []
    count =0
    ageDistribution = {}
    allAgeGroups = []
    results = session.query(nba.AGE, nba.PLAYER).filter(nba.Season == year, nba.Type ==typ)\
                .order_by(nba.AGE.desc()).all()
    #print(len(results))
    #print(results[0])
    for i in range (0, len(results)-1):   
        if(results[i][0] ==results[i+1][0]):
            names.append(results[i][1])
            #count = count +1
            #print(names)
        else:
            names.append(results[i][1])
            age=results[i][0]
            count = len(names)
            #print(count)
            ageDistribution ={age : [names, count]}
            allAgeGroups.append(ageDistribution)
            age =0
            count = 0
            names = []
    names.append(results[i+1][1])
    age=results[i][0]
    count = len(names)
    ageDistribution ={age : [names, count]}
    allAgeGroups.append(ageDistribution)
    #print(allAgeGroups)
    return allAgeGroups 
    
ageDistribution(2016, 'Regular')

[{40: [['Andre Miller', 'Kevin Garnett', 'Tim Duncan'], 3]},
 {39: [['Pablo Prigioni', 'Vince Carter'], 2]},
 {38: [['Dirk Nowitzki', 'Jason Terry', 'Manu Ginobili', 'Paul Pierce'], 4]},
 {37: [['Chris Andersen', 'Elton Brand', 'Kobe Bryant'], 3]},
 {36: [['Caron Butler',
    'Jamal Crawford',
    'Luis Scola',
    'Matt Barnes',
    'Matt Bonner',
    'Metta World Peace',
    'Mike Miller',
    'Richard Jefferson',
    'Steve Blake',
    'Tayshaun Prince',
    'Udonis Haslem'],
   11]},
 {35: [['David West',
    'James Jones',
    'Joe Johnson',
    'Kirk Hinrich',
    'Kyle Korver',
    'Mike Dunleavy',
    'Nick Collison',
    'Pau Gasol'],
   8]},
 {34: [['Boris Diaw',
    'Chris Kaman',
    'Drew Gooden',
    'Dwyane Wade',
    'Jameer Nelson',
    'Jose Calderon',
    'Tony Allen',
    'Tony Parker',
    'Zach Randolph'],
   9]},
 {33: [['Alan Anderson',
    "Amar'e Stoudemire",
    'Anderson Varejao',
    'Beno Udrih',
    'Channing Frye',
    'David Lee',
    'Devin Harris',
  

In [36]:
#age and 3pointers of all players in a given year 
year = 2017
type = 'Regular'
#@app.route('/ageAnd3Pts/<year>/<type>')
def ageAnd3Pts(year, typ):
    age =0
    names = []
    THREEPM = []
    age3PMDistribution = {}
    allAge3PMGroups = []
    results = session.query(nba.AGE, nba.PLAYER, nba.THREEPM).filter(nba.Season == year, nba.Type ==typ)\
                .order_by(nba.AGE.desc()).all()
    #print(len(results))
    #print(results[0])
    for i in range (0, len(results)-1):   
        if(results[i][0] ==results[i+1][0]):
            names.append(results[i][1])
            THREEPM.append(results[i][2])
            #print(names)
        else:
            names.append(results[i][1])
            THREEPM.append(results[i][2])
            age=results[i][0]
            age3PMDistribution ={age : [names, THREEPM]}
            allAge3PMGroups.append(age3PMDistribution)
            age =0
            names = []
            THREEPM = []
    names.append(results[i+1][1])
    THREEPM.append(results[i+1][2])
    age=results[i][0]
    age3PMDistribution ={age : [names, THREEPM]}
    allAge3PMGroups.append(age3PMDistribution)
    #print(allAgeGroups)
    return allAge3PMGroups 
    
ageAnd3Pts(2016, 'Regular')

[{40: [['Andre Miller', 'Kevin Garnett', 'Tim Duncan'], [2, 0, 0]]},
 {39: [['Pablo Prigioni', 'Vince Carter'], [28, 59]]},
 {38: [['Dirk Nowitzki', 'Jason Terry', 'Manu Ginobili', 'Paul Pierce'],
   [126, 93, 70, 75]]},
 {37: [['Chris Andersen', 'Elton Brand', 'Kobe Bryant'], [4, 0, 133]]},
 {36: [['Caron Butler',
    'Jamal Crawford',
    'Luis Scola',
    'Matt Barnes',
    'Matt Bonner',
    'Metta World Peace',
    'Mike Miller',
    'Richard Jefferson',
    'Steve Blake',
    'Tayshaun Prince',
    'Udonis Haslem'],
   [3, 117, 65, 119, 15, 26, 19, 66, 52, 4, 1]]},
 {35: [['David West',
    'James Jones',
    'Joe Johnson',
    'Kirk Hinrich',
    'Kyle Korver',
    'Mike Dunleavy',
    'Nick Collison',
    'Pau Gasol'],
   [3, 41, 120, 24, 158, 39, 0, 24]]},
 {34: [['Boris Diaw',
    'Chris Kaman',
    'Drew Gooden',
    'Dwyane Wade',
    'Jameer Nelson',
    'Jose Calderon',
    'Tony Allen',
    'Tony Parker',
    'Zach Randolph'],
   [25, 1, 7, 7, 43, 84, 15, 27, 6]]},
 {33:

In [39]:
#age and top 3points of all players in a given year 
year = 2017
type = 'Regular'
#@app.route('/ageAndTop3Pts/<year>/<type>')
def ageAndTop3Pts(year, typ):
    age =[]
    TopTHREEPM = []
    THREEPM = []
    ageTop3PMDistribution = {}
    results = session.query(nba.AGE, nba.THREEPM).filter(nba.Season == year, nba.Type ==typ)\
                .order_by(nba.AGE.desc()).all()
    for i in range (0, len(results)-1):   
        if(results[i][0] ==results[i+1][0]):
            THREEPM.append(results[i][1])
        else:
            THREEPM.append(results[i][1])
            age.append(results[i][0])
            TopTHREEPM.append(max(THREEPM))
            THREEPM = []
    THREEPM.append(results[i+1][1])
    TopTHREEPM.append(max(THREEPM))
    age.append(results[i][0])
    ageTop3PMDistribution = dict(zip(age, TopTHREEPM))
    return (ageTop3PMDistribution)
    
ageAndTop3Pts(2016, 'Playoff')

{20: 6,
 21: 2,
 22: 17,
 23: 15,
 24: 51,
 25: 42,
 26: 98,
 27: 46,
 28: 80,
 29: 26,
 30: 65,
 31: 32,
 32: 25,
 33: 26,
 34: 12,
 35: 24,
 36: 11,
 37: 0,
 38: 9,
 39: 7,
 40: 1}

In [ ]:
#age and FGM of all players in a given year 
year = 2017
type = 'Regular'
#@app.route('/ageAndFGM/<year>/<type>')
def ageAndFGM(year, typ):
    age =0
    names = []
    FGM = []
    ageFGMDistribution = {}
    allAgeFGMGroups = []
    results = session.query(nba.AGE, nba.PLAYER, nba.FGM).filter(nba.Season == 2017, nba.Type =="Regular")\
                .order_by(nba.AGE.desc()).all()
    #print(len(results))
    #print(results[0])
    for i in range (0, len(results)-1):   
        if(results[i][0] ==results[i+1][0]):
            names.append(results[i][1])
            FGM.append(results[i][2])
            #print(names)
        else:
            names.append(results[i][1])
            FGM.append(results[i][2])
            age=results[i][0]
            ageFGMDistribution ={age : [names, FGM]}
            allAgeFGMGroups.append(ageFGMDistribution)
            age =0
            names = []
            FGM = []
    names.append(results[i+1][1])
    FGM.append(results[i+1][2])
    age=results[i][0]
    ageFGMDistribution ={age : [names, FGM]}
    allAgeFGMGroups.append(ageFGMDistribution)
    #print(allAgeGroups)
    return allAgeFGMGroups 
    
ageAndFGM()

In [15]:
#age and top FGM of all players in a given year 
year = 2017
type = 'Regular'
#@app.route('/ageAndTopFGM/<year>/<type>')
def ageAndTopFGM(year, typ):
    age =[]
    TopFGM = []
    FGM = []
    ageTopFGMDistribution = {}
    results = session.query(nba.AGE, nba.FGM).filter(nba.Season == 2017, nba.Type =="Regular")\
                .order_by(nba.AGE.desc()).all()
    for i in range (0, len(results)-1):   
        if(results[i][0] == results[i+1][0]):
            FGM.append(results[i][1])
        else:
            FGM.append(results[i][1])
            age.append(results[i][0])
            TopFGM.append(max(FGM))
            FGM = []
    FGM.append(results[i+1][1])
    TopFGM.append(max(FGM))
    age.append(results[i][0])
    ageTopFGMDistribution = dict(zip(age, TopFGM))
    return (ageTopFGMDistribution)
    
ageAndTopFGM()

{19: 284,
 20: 606,
 21: 802,
 22: 709,
 23: 548,
 24: 770,
 25: 692,
 26: 661,
 27: 721,
 28: 824,
 29: 675,
 30: 433,
 31: 534,
 32: 736,
 33: 602,
 34: 248,
 35: 433,
 36: 303,
 37: 359,
 39: 296,
 40: 193}

In [ ]:
#age and %Winning of all players in a given year 
year = 2017
type = 'Regular'
#@app.route('/ageAndPWin/<year>/<type>')
def ageAndPWin(year, typ):
    age =0
    names = []
    win = 0
    totalGames = 0
    PWin = []
    agePWinDistribution = {}
    allAgePWinGroups = []
    results = session.query(nba.AGE, nba.PLAYER, nba.W, nba.GP).filter(nba.Season == 2017, nba.Type =="Regular")\
                .order_by(nba.AGE.desc()).all()
    #print(len(results))
    #print(results[0])
    for i in range (0, len(results)-1):   
        if(results[i][0] ==results[i+1][0]):
            names.append(results[i][1])
            win = round(100*(results[i][2]/results[i][3]),2)
            PWin.append(win)
            #print(names)
        else:
            names.append(results[i][1])
            win = round(100*(results[i][2]/results[i][3]),2)
            PWin.append(win)
            age=results[i][0]
            agePWinDistribution ={age : [names, PWin]}
            allAgePWinGroups.append(agePWinDistribution)
            age = 0
            win = 0
            names = []
            PWin = []
    names.append(results[i+1][1])
    win = round(100*(results[i+1][2]/results[i+1][3]), 2)
    PWin.append(win)
    age=results[i][0]
    agePWinDistribution ={age : [names, PWin]}
    allAgePWinGroups.append(agePWinDistribution)
    #print(allAgeGroups)
    return allAgePWinGroups 
    
ageAndPWin()

In [24]:
#age and top PWin of all players in a given year 
year = 2017
typ = 'Regular'
#@app.route('/ageAndTopPWin/<year>/<typ>')
def ageAndTopPWin(year, typ):
    age = []
    win = 0
    TopPwin = []
    PWin = []
    ageTopPwinDistribution = {}
    results = session.query(nba.AGE, nba.W, nba.GP).filter(nba.Season == year, nba.Type ==typ)\
                .order_by(nba.AGE.desc()).all()
    for i in range (0, len(results)-1):   
        if(results[i][0] == results[i+1][0]):
            win = round(100*(results[i][1]/results[i][2]),2)
            PWin.append(win)
        else:
            win = round(100*(results[i][1]/results[i][2]),2)
            PWin.append(win)
            age.append(results[i][0])
            TopPwin.append(max(PWin))
            win = 0
            PWin = []
    win = round(100*(results[i+1][1]/results[i+1][2]),2)
    PWin.append(win)
    TopPwin.append(max(PWin))
    age.append(results[i][0])
    ageTopPwinDistribution = dict(zip(age, TopPwin))
    return (ageTopPwinDistribution)

ageAndTopPWin(2017, 'Regular')

{19: 32.91,
 20: 76.32,
 21: 83.02,
 22: 70.0,
 23: 73.61,
 24: 85.71,
 25: 70.37,
 26: 81.82,
 27: 84.62,
 28: 82.26,
 29: 82.28,
 30: 75.0,
 31: 82.89,
 32: 72.0,
 33: 84.21,
 34: 78.95,
 35: 73.02,
 36: 82.35,
 37: 63.29,
 39: 73.91,
 40: 49.32}